In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load user data
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

# Load ratings data
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

# Load items data
i_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date',
          'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's",
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1', usecols=range(24))

# Preprocess item data - TF-IDF on genres
transformer = TfidfTransformer(smooth_idf=True, norm='l2')
tfidf = transformer.fit_transform(items.iloc[:, 5:].fillna(0)).toarray()

# Merge ratings with user and item data for more features
data = ratings.join(users.set_index('user_id'), on='user_id')
data = data.join(items.set_index('movie_id'), on='movie_id')

# Normalize ratings
scaler = StandardScaler()
data['normalized_rating'] = scaler.fit_transform(data[['rating']])

# Setup cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []
accuracy_scores = []

for train_index, test_index in kf.split(data):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    # Train the model
    clf = Ridge(alpha=0.01)
    clf.fit(tfidf[train_data['movie_id'] - 1], train_data['normalized_rating'])

    # Predict on test set
    preds = clf.predict(tfidf[test_data['movie_id'] - 1])

    # Reverse normalization for true ratings and calculate RMSE
    preds_denorm = scaler.inverse_transform(preds.reshape(-1, 1))
    rmse = sqrt(mean_squared_error(test_data['rating'], preds_denorm))
    rmse_scores.append(rmse)

    # Calculate custom accuracy
    accuracy = np.mean(np.abs(preds_denorm.ravel() - test_data['rating'].values) <= 0.5)
    accuracy_scores.append(accuracy)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f"Mean RMSE: {mean_rmse:.3f} ± {std_rmse:.3f}")
print(f"Accuracy (within 0.5 of true rating): {mean_accuracy:.3f} ± {std_accuracy:.3f}")


Mean RMSE: 1.108 ± 0.005
Accuracy (within 0.5 of true rating): 0.335 ± 0.003
